In [22]:
import re
import time
from glob import glob
from tqdm import tqdm

import pandas as pd
import datetime
from pandas import DataFrame
from datetime import timedelta


import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

In [3]:
df_ipo = pd.read_csv('../data/IPO_basic_information.csv',encoding='cp949',index_col=0)

In [4]:
df_ipo.head()

,기업명,시장구분,업종,매출액,순이익,자본금,종목코드,기업구분,확정공모가,주간사,수요예측일,공모청약일,상장일,기관경쟁률,의무보유확약
0,바이오노트,거래소,의료용품 및 기타 의약 관련제품 제조업,622355,671556,1266407,377740,중소일반,9000,NH투자증권,2022.12.08 ~ 2022.12.09,2022.12.13 ~ 2022.12.14,2022.12.22,3.29,2.77
1,에스에이엠지엔터테인먼트,코스닥,"영화, 비디오물, 방송프로그램 제작 및 배급업",38374,-11087,4295,419530,중소일반,17000,NH투자증권,2022.11.17 ~ 2022.11.18,2022.11.24 ~ 2022.11.25,2022.12.06,44.73,0.05
2,펨트론,코스닥,기타 전기장비 제조업,57871,4499,5322,168360,중소일반,8000,하나증권,2022.11.08 ~ 2022.11.09,2022.11.15 ~ 2022.11.16,2022.11.24,129.06,11.41
3,인벤티지랩,코스닥,의약품 제조업,1862,-9271,4102,389470,중소일반,12000,한국투자증권,2022.11.08 ~ 2022.11.09,2022.11.11 ~ 2022.11.14,2022.11.22,14.37,0.00
4,엔젯,코스닥,평판디스플레이 제조용 기계 제조업,10055,-29867,5258,419080,중소일반,10000,미래에셋증권,2022.11.03 ~ 2022.11.04,2022.11.09 ~ 2022.11.10,2022.11.18,42.15,6.00


In [21]:
for ind in df_ipo.query('기업명.str.contains("구.")').index.to_list():
    l = df_ipo['기업명'][ind].find('(구.')+3
    df_ipo['기업명'][ind] = df_ipo['기업명'][ind][l:-1]

/var/folders/jx/rdhgw4n94cvb3tl_p0k9_4br0000gn/T/ipykernel_1649/567256607.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ipo['기업명'][ind] = df_ipo['기업명'][ind][l:-1]


In [23]:
IPO_news = []
relative_news = []
for ind,IPO_name in tqdm(enumerate(df_ipo['기업명'])):
    behind_date_1 = df_ipo['상장일'][ind]
    behind_date_2 = behind_date_1.replace('.','')
    behind_year = int(behind_date_1[:4])
    behind_month = int(behind_date_1[5:7])
    behind_day = int(behind_date_1[8:10])
    behind_date = (datetime.datetime(behind_year,behind_month,behind_day) - timedelta(days=1)).strftime("%Y.%m.%d")
    behind_date_0 = behind_date.replace('.','')
    front_date_1 = (datetime.datetime(behind_year,behind_month,behind_day)- timedelta(days=30)).strftime("%Y.%m.%d")
    front_date_2 = front_date_1.replace('.','')  
    page_num = 1
#-------------------------------------------------------------------------------
    while True:
        url = f'https://search.naver.com/search.naver?where=news&query={IPO_name}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={front_date_1}&de={behind_date}&from{front_date_2}to{behind_date_0}&is_sug_officeid=&nso=so:r,p:all,a:all&start={page_num}'
        response = requests.get(url,headers={'User-Agent':'Mozilla/5.0'})
        html = response.text
        p_soup = BeautifulSoup(html, 'html.parser')
        page_article = len(p_soup.select('#main_pack > section > div > div.group_news > ul > li > div > div > div > div.info_group'))
        if len(p_soup.select('#main_pack > section > div > div.group_news > ul > li > a')) != 0:
            for value in range(len(p_soup.select('#main_pack > section > div > div.group_news > ul > li > a'))):
                relative_news.append([IPO_name,p_soup.select('#main_pack > section > div > div.group_news > ul > li > a')[value].attrs['href']])
        for article in range(page_article):
            try:
                n_url = p_soup.select('#main_pack > section > div > div.group_news > ul > li > div > div > div > div.info_group')[article].findAll()[-1].attrs['href']
                response = requests.get(n_url,headers={'User-Agent':'Mozilla/5.0'})
                html = response.text
                soup = BeautifulSoup(html, 'html.parser')
                healine = soup.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_title > h2').get_text()
                body_text = soup.select_one('#dic_area').get_text().replace('\n','')
                media_coverage = soup.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_top > a').attrs['href']
                IPO_news.append([IPO_name,media_coverage,healine,body_text])        
            except:
                continue
        if type(p_soup.select_one("a.btn_next")) == 'NoneType':
            break
        elif p_soup.select_one("a.btn_next").get('aria-disabled') == 'false':
            page_num += 10
        else:
            break

20it [05:48, 17.41s/it]

KeyboardInterrupt



In [12]:
url

'https://search.naver.com/search.naver?where=news&query=하이브(구.빅히트)&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2020.09.15&de=2020.10.14&from20200915to20201014&is_sug_officeid=&nso=so:r,p:all,a:all&start=1'